## Classification of Income Being Above or Below/Equal to $50,000 in Adult Census Data

By: Sunsar, Sarah, Emily, Calvin (DSCI 100 003 - Group 23)

Data is from: https://www.kaggle.com/datasets/uciml/adult-census-income

The dataset used for this analysis is derived from the 1994 Census Bureau database. 

The dataset contains a diverse range of numerical and categorical attributes, such as age, hours worked per week,  sex, and more. In this project, we will filter and simplify some categories from this dataset to predict whether an individual"s annual salary falls above or below $50,000.

The question this project will seek to answer is: 

**How do different aspects of a person predict whether annual income will be above or below $50K annually?**

# Reading in Data

In [8]:
library(tidyverse)
library(tidymodels)

In [9]:
df = read_csv("https://raw.githubusercontent.com/calvingdu/dsci100-003-23/master/data/adult_census.csv")

# Splitting the data
df_split <- initial_split(df, prop = 0.8, strata = income)
df_train <- training(df_split)
df_test <- testing(df_split)

paste0("Training set row count: ", nrow(df_train))
paste0("Testing set row count: ", nrow(df_test))

Rows: 32561 Columns: 15
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (9): workclass, education, marital.status, occupation, relationship, rac...
dbl (6): age, fnlwgt, education.num, capital.gain, capital.loss, hours.per.week

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "Training set row count: 26048"

[1] "Testing set row count: 6513"

In [10]:
head(df_train, 3)

age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


# Tidying/Cleaning The Data & Selecting Columns
We can begin by tidying up the data. Something we noticed immediately that there are a lot of categorical columns. To tackle this, we plan to make buckets of categories and then turn them into dummy variables. For example, we can turn __sex__ into a dummy variable where 0 = male and 1 = female, make dummy variables by bucketting different things such as paid/unpaid in __workclass__, or making a dummy variable for every unique value. 

These are the columns we plan to use and how we plan to tackle them to make them usable in data: 
- **Age**: No changes (either than scaling/imputation)
- **Workclass**: Make a dummy variable for paid/unpaid
- **Education**: Simplified to a dummy variable of if College Graduate or Not
- **Occupation**: Make a dummy variable for all the occupations
- **Relationship**: Simplified a dummy variable of Married or Not
- **Sex**: Transformed into a Dummy Variable
- **Capital Gain**: No changes (either than scaling/imputation)
- **Capital Loss**: No changes (either than scaling/imputation)
- **Hours Per Week**: No changes (either than scaling/imputation)
- **Native Country**: Evaluated what country has the most people with income > 50k (United States by a large margin) and then make a dummy variable for being in this country or not 

**Dropped Columns & Reasoning**: 
- fnlwgt: Unclear how this relates
- education.num: Already have education
- race: Don"t believe this is significant to the study so removing to avoid overfitting 
- marital.status: We think it"s similar to relationship so we remove to avoid overfitting 


This is also some tidying to do. We can initially see that there are some missing values in __workclass__ and __occupation__ represented as ?. Since we believe these are extremely important roles to guess income category, we remove any rows that don't have data for this. 

In [11]:
filtered_df <- df_train |>
  select(age, workclass, education, occupation, relationship, sex,
         capital.gain, capital.loss, hours.per.week, native.country, income) |>
  filter(workclass != "?" & occupation != "?")

Then, we can begin by making dummy variables in the data using the above choices

### Binary Preprocessing

In [12]:
# Making a function to be used later on testing data 
preprocess_binary <- function(df){
    transformed_df <- df |>
        mutate(is_married = ifelse((relationship == "Husband" | relationship == "Wife" |relationship == "Wife"),1,0)) |>
        mutate(sex_dummy = ifelse(sex == "Female", 1, 0)) |>
        mutate(being_paid = ifelse(workclass != "Self-emp-not-inc" & workclass != "Without-pay" & workclass != "Never-worked", 1, 0)) |>
        mutate(is_american = ifelse(native.country == "United-States", 1, 0))
    return(transformed_df)
}

binary_df <- preprocess_binary(filtered_df)

### Dummy Variables for Every Occupation

In [13]:
unique_occupations <- as.list(unique(binary_df$occupation))

# Making a function to use later
preprocess_occupation <- function(df, occupations){
    for (occ in occupations) {
        col_name <- gsub("[^a-zA-Z0-9 ]", ".", tolower(occ))
        df[[col_name]] <- as.integer(df$occupation == occ)
    }
    return(df)
}

occupations_df <- preprocess_occupation(binary_df, unique_occupations)

### Removing Original Variables

In [14]:
tidy_df <- occupations_df |>
    select(-c(occupation, workclass, education, occupation, relationship, sex, native.country))

head(tidy_df)

age,capital.gain,capital.loss,hours.per.week,income,is_married,sex_dummy,being_paid,is_american,exec.managerial,...,sales,transport.moving,craft.repair,farming.fishing,adm.clerical,handlers.cleaners,protective.serv,tech.support,priv.house.serv,armed.forces
<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
82,0,4356,18,<=50K,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
54,0,3900,40,<=50K,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
41,0,3900,40,<=50K,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
34,0,3770,45,<=50K,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
68,0,3683,40,<=50K,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
61,0,2754,25,<=50K,0,1,1,1,0,...,1,0,0,0,0,0,0,0,0,0


In [17]:
process_df <- function(df){
    filtered_df <- df |>
    select(age, workclass, education, occupation, relationship, sex,
            capital.gain, capital.loss, hours.per.week, native.country, income) |>
    filter(workclass != "?" & occupation != "?")

    binary_df <- preprocess_binary(filtered_df)

    unique_occupations <- as.list(unique(binary_df$occupation))
    occupations_df <- preprocess_occupation(binary_df, unique_occupations) 
    new_cols_df <- select(occupations_df, -c(occupation, workclass, education, occupation, relationship, sex, native.country))

    return(new_cols_df)
}

tidy_df <- process_df(df_train)

# Preliminary Exploratory Data Analysis



## Summary Table

In [16]:
tidy_df |>
group_by(income) |>
summarize(across(everything(), ~ mean(., na.rm = TRUE), .names = "mean_{.col}")) |>
t()

income,<=50K,>50K
mean_age,36.56475,43.98363
mean_capital.gain,150.4734,4107.0524
mean_capital.loss,53.08029,192.70933
mean_hours.per.week,39.41570,45.82488
mean_is_married,0.3290742,0.8454992
mean_sex_dummy,0.3829995,0.1487725
mean_being_paid,0.9206822,0.9083470
mean_is_american,0.8884136,0.9147300
mean_exec.managerial,0.09057932,0.25433715
mean_machine.op.inspct,0.07498646,0.03109656


## Summary Visualization

# Methods

# Expected outcomes and significance